In [43]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import warnings
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from catboost import CatBoostClassifier
import seaborn as sns
import joblib

In [3]:
train = pd.read_csv("train_ctrUa4K.csv")
test = pd.read_csv("test_lAUu6dG.csv")


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


In [5]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367 entries, 0 to 366
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            367 non-null    object 
 1   Gender             356 non-null    object 
 2   Married            367 non-null    object 
 3   Dependents         357 non-null    object 
 4   Education          367 non-null    object 
 5   Self_Employed      344 non-null    object 
 6   ApplicantIncome    367 non-null    int64  
 7   CoapplicantIncome  367 non-null    int64  
 8   LoanAmount         362 non-null    float64
 9   Loan_Amount_Term   361 non-null    float64
 10  Credit_History     338 non-null    float64
 11  Property_Area      367 non-null    object 
dtypes: float64(3), int64(2), object(7)
memory usage: 34.5+ KB


In [6]:
train['Loan_ID'].nunique

<bound method IndexOpsMixin.nunique of 0      LP001002
1      LP001003
2      LP001005
3      LP001006
4      LP001008
         ...   
609    LP002978
610    LP002979
611    LP002983
612    LP002984
613    LP002990
Name: Loan_ID, Length: 614, dtype: object>

In [7]:
test['Loan_ID'].nunique

<bound method IndexOpsMixin.nunique of 0      LP001015
1      LP001022
2      LP001031
3      LP001035
4      LP001051
         ...   
362    LP002971
363    LP002975
364    LP002980
365    LP002986
366    LP002989
Name: Loan_ID, Length: 367, dtype: object>

In [8]:
train.head().T


,0,1,2,3,4
Loan_ID,LP001002,LP001003,LP001005,LP001006,LP001008
Gender,Male,Male,Male,Male,Male
Married,No,Yes,Yes,Yes,No
Dependents,0,1,0,0,0
Education,Graduate,Graduate,Graduate,Not Graduate,Graduate
Self_Employed,No,No,Yes,No,No
ApplicantIncome,5849,4583,3000,2583,6000
CoapplicantIncome,0,1508,0,2358,0
LoanAmount,NaN,128,66,120,141
Loan_Amount_Term,360,360,360,360,360


In [9]:
test.head().T



,0,1,2,3,4
Loan_ID,LP001015,LP001022,LP001031,LP001035,LP001051
Gender,Male,Male,Male,Male,Male
Married,Yes,Yes,Yes,Yes,No
Dependents,0,1,2,2,0
Education,Graduate,Graduate,Graduate,Graduate,Not Graduate
Self_Employed,No,No,No,No,No
ApplicantIncome,5720,3076,5000,2340,3276
CoapplicantIncome,0,1500,1800,2546,0
LoanAmount,110,126,208,100,78
Loan_Amount_Term,360,360,360,360,360


In [10]:
train.isnull().sum()




Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [11]:
train['Gender'].fillna(train['Gender'].mode()[0], inplace=True)

train['Married'].fillna(train['Married'].mode()[0], inplace=True)
train['Dependents'].fillna(train['Dependents'].mode()[0], inplace=True)
train['Self_Employed'].fillna(train['Self_Employed'].mode()[0], inplace=True)
train['Credit_History'].fillna(train['Credit_History'].mode()[0], inplace=True)

In [12]:
train.isnull().sum()

Loan_ID               0
Gender                0
Married               0
Dependents            0
Education             0
Self_Employed         0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History        0
Property_Area         0
Loan_Status           0
dtype: int64

In [13]:
train['Loan_Amount_Term'].fillna(train['Loan_Amount_Term'].mode()[0], inplace=True)
train['LoanAmount'].fillna(train['LoanAmount'].median(), inplace=True)

In [14]:

train.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [15]:
test.isnull().sum()


Loan_ID               0
Gender               11
Married               0
Dependents           10
Education             0
Self_Employed        23
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            5
Loan_Amount_Term      6
Credit_History       29
Property_Area         0
dtype: int64

In [16]:
test['Gender'].fillna(test['Gender'].mode()[0], inplace=True)
test['Dependents'].fillna(test['Dependents'].mode()[0], inplace=True)
test['Self_Employed'].fillna(test['Self_Employed'].mode()[0], inplace=True)
test['Credit_History'].fillna(test['Credit_History'].mode()[0], inplace=True)
test['Loan_Amount_Term'].fillna(test['Loan_Amount_Term'].mode()[0], inplace=True)
test['LoanAmount'].fillna(test['LoanAmount'].median(), inplace=True)

In [17]:
test.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
dtype: int64

In [18]:
cat_cols_train = [col for col in train.columns if train[col].dtype == 'object']

cat_cols_test = [col for col in test.columns if test[col].dtype == 'object']

num_cols_train = [col for col in train.columns if train[col].dtype != 'object']

num_cols_test = [col for col in test.columns if test[col].dtype != 'object']

In [19]:
cat_cols_train

['Loan_ID',
 'Gender',
 'Married',
 'Dependents',
 'Education',
 'Self_Employed',
 'Property_Area',
 'Loan_Status']

In [20]:
cat_cols_test

['Loan_ID',
 'Gender',
 'Married',
 'Dependents',
 'Education',
 'Self_Employed',
 'Property_Area']

In [21]:
num_cols_train

['ApplicantIncome',
 'CoapplicantIncome',
 'LoanAmount',
 'Loan_Amount_Term',
 'Credit_History']

In [22]:
num_cols_test

['ApplicantIncome',
 'CoapplicantIncome',
 'LoanAmount',
 'Loan_Amount_Term',
 'Credit_History']

In [23]:
cat_cols_train.remove('Loan_ID')
cat_cols_train.remove('Loan_Status')
cat_cols_test.remove('Loan_ID')

In [24]:
cat_cols_train

['Gender',
 'Married',
 'Dependents',
 'Education',
 'Self_Employed',
 'Property_Area']

In [25]:
cat_cols_test

['Gender',
 'Married',
 'Dependents',
 'Education',
 'Self_Employed',
 'Property_Area']

In [26]:
train['Loan_Status'].unique()

array(['Y', 'N'], dtype=object)

In [27]:
train['Loan_Status'] = train['Loan_Status'].map({'Y': 1, 'N': 0})

In [28]:
train['Loan_Status'].unique()

array([1, 0])

In [29]:
def rate_encoder(df, col):
    rate = train.groupby(col)['Loan_Status'].sum()/train.groupby(col)['Loan_Status'].count()
    rate_dict = dict(rate)
    df[col] = df[col].replace(rate_dict)

In [30]:
for col in cat_cols_train:
    rate_encoder(train, col)

In [31]:
for col in cat_cols_test:
    rate_encoder(test, col)

In [32]:
train[cat_cols_train]

,Gender,Married,Dependents,Education,Self_Employed,Property_Area
0,0.691235,0.629108,0.686111,0.708333,0.687970,0.658416
1,0.691235,0.718204,0.647059,0.708333,0.687970,0.614525
2,0.691235,0.718204,0.686111,0.708333,0.682927,0.658416
3,0.691235,0.718204,0.686111,0.611940,0.687970,0.658416
4,0.691235,0.629108,0.686111,0.708333,0.687970,0.658416
...,...,...,...,...,...,...
609,0.669643,0.629108,0.686111,0.708333,0.687970,0.614525
610,0.691235,0.718204,0.647059,0.708333,0.687970,0.614525
611,0.691235,0.718204,0.647059,0.708333,0.687970,0.658416
612,0.691235,0.718204,0.752475,0.708333,0.687970,0.658416


In [33]:
train[cat_cols_train + ['Loan_Status']].corr()

,Gender,Married,Dependents,Education,Self_Employed,Property_Area,Loan_Status
Gender,1.000000,0.364569,0.066963,-0.045364,0.000525,-0.112640,0.017987
Married,0.364569,1.000000,0.092012,-0.012304,-0.004489,0.006817,0.091478
Dependents,0.066963,0.092012,1.000000,-0.003327,0.012942,-0.019660,0.071642
Education,-0.045364,-0.012304,-0.003327,1.000000,-0.010383,0.055580,0.085884
Self_Employed,0.000525,-0.004489,0.012942,-0.010383,1.000000,-0.000366,0.003700
Property_Area,-0.112640,0.006817,-0.019660,0.055580,-0.000366,1.000000,0.141523
Loan_Status,0.017987,0.091478,0.071642,0.085884,0.003700,0.141523,1.000000


In [34]:
train[num_cols_train + ['Loan_Status']].corr()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status
ApplicantIncome,1.000000,-0.116605,0.565181,-0.046531,-0.018615,-0.004710
CoapplicantIncome,-0.116605,1.000000,0.189218,-0.059383,0.011134,-0.059187
LoanAmount,0.565181,0.189218,1.000000,0.036960,-0.000607,-0.033214
Loan_Amount_Term,-0.046531,-0.059383,0.036960,1.000000,-0.004705,-0.022549
Credit_History,-0.018615,0.011134,-0.000607,-0.004705,1.000000,0.540556
Loan_Status,-0.004710,-0.059187,-0.033214,-0.022549,0.540556,1.000000


In [35]:
features =  num_cols_train + cat_cols_train

In [36]:
features

['ApplicantIncome',
 'CoapplicantIncome',
 'LoanAmount',
 'Loan_Amount_Term',
 'Credit_History',
 'Gender',
 'Married',
 'Dependents',
 'Education',
 'Self_Employed',
 'Property_Area']

In [37]:
X = train[features]
y = train.Loan_Status
X_test = test[features]

In [38]:
split_test_size=0.3

Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, y, test_size=split_test_size, random_state=0)

clf = CatBoostClassifier(iterations=3000, depth=3, eval_metric='Accuracy', random_seed=10, learning_rate=0.1)

clf.fit(Xtrain, Ytrain, use_best_model=True, eval_set=(Xtest,Ytest), verbose=1000) 

0:	learn: 0.8018648	test: 0.8270270	best: 0.8270270 (0)	total: 56.2ms	remaining: 2m 48s
1000:	learn: 1.0000000	test: 0.7945946	best: 0.8324324 (95)	total: 692ms	remaining: 1.38s
2000:	learn: 1.0000000	test: 0.7945946	best: 0.8324324 (95)	total: 1.31s	remaining: 656ms
2999:	learn: 1.0000000	test: 0.7891892	best: 0.8324324 (95)	total: 1.9s	remaining: 0us

bestTest = 0.8324324324
bestIteration = 95

Shrink model to first 96 iterations.


In [39]:
model = CatBoostClassifier(iterations=96, depth=3, eval_metric='Accuracy', random_seed=10, learning_rate=0.1)

In [40]:
model.fit(X,y)

0:	learn: 0.8094463	total: 1.22ms	remaining: 116ms
1:	learn: 0.8094463	total: 2.25ms	remaining: 106ms
2:	learn: 0.8094463	total: 3.2ms	remaining: 99.3ms
3:	learn: 0.8094463	total: 3.82ms	remaining: 88ms
4:	learn: 0.8094463	total: 4.71ms	remaining: 85.7ms
5:	learn: 0.8094463	total: 5.47ms	remaining: 82.1ms
6:	learn: 0.8094463	total: 6.38ms	remaining: 81.1ms
7:	learn: 0.8094463	total: 7.3ms	remaining: 80.3ms
8:	learn: 0.8094463	total: 8.7ms	remaining: 84.1ms
9:	learn: 0.8094463	total: 9.44ms	remaining: 81.2ms
10:	learn: 0.8094463	total: 10.1ms	remaining: 78.2ms
11:	learn: 0.8094463	total: 10.6ms	remaining: 74.2ms
12:	learn: 0.8094463	total: 11.1ms	remaining: 71.2ms
13:	learn: 0.8094463	total: 11.8ms	remaining: 69.1ms
14:	learn: 0.8094463	total: 12.4ms	remaining: 67.1ms
15:	learn: 0.8094463	total: 13.5ms	remaining: 67.4ms
16:	learn: 0.8094463	total: 14.2ms	remaining: 66.2ms
17:	learn: 0.8094463	total: 14.8ms	remaining: 64ms
18:	learn: 0.8094463	total: 15.3ms	remaining: 62ms
19:	learn: 0.8

In [45]:
joblib.dump(model, 'model.pkl')

['model.pkl']

In [46]:
model = joblib.load('model.pkl')

In [47]:
model_columns = list(X.columns)

In [48]:
joblib.dump(model_columns, 'model_columns.pkl')

['model_columns.pkl']